In [1]:
import torch.nn as nn
import torch.optim as optim
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from ast import literal_eval
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "9"

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
num_workers=1
batch_size=1000
lr=0.01
num_epoch=200

In [2]:
num_level=1
depth_batch=1
hide_neuron=32

annotation_train = 'data/data_se/data_train.txt'
annotation_val = 'data/data_se/data_test.txt'
var_size=1370+1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
class MyDataset(Dataset):
    def __init__(self, annotation):
        self.landmarks_frame = pd.read_csv(annotation)
    def __len__(self):
        return len(self.landmarks_frame)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        i_data = literal_eval(self.landmarks_frame.iloc[idx, 1])

        return i_data
train_dataset = MyDataset(annotation_train)
val_dataset = MyDataset(annotation_val)

In [4]:
def pad_length(b):
    # build vectorized sequence
    v = b
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=var_size-1) for t in v]),
    )

In [5]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True, drop_last=True, shuffle=True,collate_fn=pad_length)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True, drop_last=True, shuffle=False,collate_fn=pad_length)

In [6]:
class MyLSTM(nn.Module):
    def __init__(self, output_size, hidden_size, n_layers=1):
        super(MyLSTM, self).__init__()
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.LSTM = nn.LSTM(self.output_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.output_size)
    def forward(self,x,hidden):
        out, (h, c)  = self.LSTM(x,hidden)
        out = self.dropout(out)
        x = self.fc(out)
        x = self.fc(out)
        return x, (h, c)

In [7]:
net=MyLSTM(var_size,hide_neuron,num_level)
net=net.to(device)
Loss = nn.CrossEntropyLoss(ignore_index=var_size-1)
optimizer = optim.AdamW(net.parameters(), lr=lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.5,patience=5)

In [8]:
def micro_batch(input,index=0,input_size=1,var_size=1):
    sting=input[:,index:index+input_size]#.type(torch.int32)
    label_ind=input[:,index+1:index+input_size+1]#.type(torch.int32)
    size=np.array([sting.shape[0]*sting.shape[1],var_size])
    batch=np.zeros(size)
    for i in range(sting.shape[0]):
        for j in range(sting.shape[1]):
            a=sting[i][j]
            batch[i+j][a]+=1.0
    label=torch.Tensor(label_ind).reshape(-1)
    batch=torch.Tensor(batch)
    return batch, label
    

In [38]:
num_epoch=80
batch_size=10

In [40]:
best_Loss_regression=np.inf
for iter in range(num_epoch):
    for dataloader, phase in zip([train_dataloader, val_dataloader], ["T", "V"]):
        eposh_loss=0
        if phase == "T": net.train()
        if phase == "V": net.eval()
        for [data] in dataloader:
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == "T"):
                total_loss = torch.tensor([0.0], requires_grad=phase == "T").to(device)
                h0 = torch.randn(num_level,  hide_neuron).to(device)
                c0 = torch.randn(num_level, hide_neuron).to(device)
                for i in range(data.shape[1]-depth_batch):
                    input_batch, target=micro_batch(data,i,depth_batch,var_size)
                    output, (h0, c0) = net(input_batch.to(device), (h0, c0))
                    target = target.long().to(device)
                    loss=Loss(output,target)
                    total_loss=total_loss+loss
                if phase == 'T':
                    # Вычислить градиенты
                    total_loss.backward()
                    # Обновить веса
                    optimizer.step()
                eposh_loss+=total_loss.item()
        if phase == "T": 
            print(f"{iter} {phase} {eposh_loss / len(dataloader):0.3f}", end=" ")
        else:
            scheduler.step(eposh_loss)
            print(f"{phase} {eposh_loss / len(dataloader):0.3f} {optimizer.param_groups[0]['lr']}")
            if eposh_loss < best_Loss_regression:
                best_model_wts = net.state_dict()
                best_model_ind = iter
                best_Loss_regression = eposh_loss
net.load_state_dict(best_model_wts)
torch.save(best_model_wts, 'model_lstm.tar')
print(best_model_ind, best_Loss_regression / len(val_dataloader))

0 T 19.771 V 20.834 0.001
1 T 19.881 V 20.831 0.001
2 T 19.988 V 20.829 0.001
3 T 19.445 V 20.886 0.001
4 T 19.348 V 20.736 0.001
5 T 19.659 V 20.766 0.001
6 T 19.285 V 20.705 0.001
7 T 19.284 V 20.784 0.001
8 T 19.431 V 20.726 0.001
9 T 19.391 V 20.697 0.001
10 T 19.393 V 20.695 0.001
11 T 19.410 V 20.640 0.001
12 T 19.182 V 20.647 0.001
13 T 19.223 V 20.651 0.001
14 T 19.359 V 20.578 0.001
15 T 19.109 V 20.574 0.001
16 T 19.096 V 20.501 0.001
17 T 19.113 V 20.676 0.001
18 T 18.915 V 20.613 0.001
19 T 18.936 V 20.568 0.001
20 T 18.685 V 20.595 0.001
21 T 18.943 V 20.547 0.001
22 T 18.871 V 20.544 0.0005
23 T 19.037 V 20.527 0.0005
24 T 18.821 V 20.458 0.0005
25 T 18.492 V 20.541 0.0005
26 T 18.996 V 20.507 0.0005
27 T 18.834 V 20.558 0.0005
28 T 18.575 V 20.523 0.0005
29 T 18.629 V 20.485 0.0005
30 T 18.837 V 20.511 0.00025
31 T 18.666 V 20.496 0.00025
32 T 18.629 V 20.491 0.00025
33 T 18.626 V 20.473 0.00025
34 T 18.437 V 20.498 0.00025
35 T 18.456 V 20.474 0.00025
36 T 18.665 V 20.4

KeyboardInterrupt: 

In [41]:
net.load_state_dict(best_model_wts)

<All keys matched successfully>

In [42]:
n = 0
trye = 0
trye_3 = 0
trye_5 = 0
trye_10 = 0
trye_50 = 0
trye_500 = 0
data_test = pd.read_csv(annotation_val)
net.eval()
for d in data_test['name_id'].values:
    mas = literal_eval(d)
    h0 = torch.randn(num_level,  hide_neuron).to(device)
    c0 = torch.randn(num_level, hide_neuron).to(device)
    label=mas[-1]
    input_v=mas[:-1]
    
    for i, m in enumerate(input_v):
        buf = np.zeros((var_size),dtype = np.float32)
        buf[m] +=1
        inpyt = torch.tensor([buf]).to(device)
        with torch.set_grad_enabled(False):
            out, (h0, c0) = net(inpyt, (h0, c0))
    trye += label == out.argmax()
    trye_3 += label in out.argsort(descending=True)[0][:3]
    trye_5 += label in out.argsort(descending=True)[0][:5]
    trye_10 += label in out.argsort(descending=True)[0][:10]
    trye_50 += label in out.argsort(descending=True)[0][:50]
    trye_500 += label in out.argsort(descending=True)[0][:500]
    n+=1
print('Acc top 1', trye / n)
print('Acc top 3',trye_3 / n)
print('Acc top 5',trye_5 / n)
print('Acc top 10',trye_10 / n)
print('Acc top 50',trye_50 / n)
print('Acc top 500',trye_500 / n)

Acc top 1 tensor(0.1648, device='cuda:0')
Acc top 3 0.3408691484262662
Acc top 5 0.43577248761783033
Acc top 10 0.5750918677104969
Acc top 50 0.8136283751397987
Acc top 500 0.9627736060073494


In [43]:
n = 0
trye = 0
trye_3 = 0
trye_5 = 0
trye_10 = 0
trye_50 = 0
trye_500 = 0
end=0
data_test = pd.read_csv(annotation_val)
net.eval()
for d in data_test['name_id'].values:
    mas = literal_eval(d)
    for i, m in enumerate(mas):
        h0 = torch.randn(num_level,  hide_neuron).to(device)
        c0 = torch.randn(num_level, hide_neuron).to(device)
        label=m
        input_v=mas[:i]+mas[i+1:]
        # print(mas,label,input_v)
        for r in input_v:
            buf = np.zeros((var_size),dtype = np.float32)
            buf[r] +=1
            inpyt = torch.tensor([buf]).to(device)
            with torch.set_grad_enabled(False):
                out, (h0, c0) = net(inpyt, (h0, c0))
        trye += label == out.argmax()
        # print(out.argsort(descending=True)[0][:3],label)
        trye_3 += label in out.argsort(descending=True)[0][:3]
        trye_5 += label in out.argsort(descending=True)[0][:5]
        trye_10 += label in out.argsort(descending=True)[0][:10]
        trye_50 += label in out.argsort(descending=True)[0][:50]
        trye_500 += label in out.argsort(descending=True)[0][:500]
        end += var_size-1 == out.argmax()
        n+=1
print('Acc top 1', trye / n)
print('Acc top 3',trye_3 / n)
print('Acc top 5',trye_5 / n)
print('Acc top 10',trye_10 / n)
print('Acc top 50',trye_50 / n)
print('Acc top 500',trye_500 / n)
# print('Acc end',end / n)

Acc top 1 tensor(0.0842, device='cuda:0')
Acc top 3 0.1876910724762993
Acc top 5 0.26183491338047743
Acc top 10 0.39727634870147915
Acc top 50 0.711607942438934
Acc top 500 0.9227063582744033


In [44]:
save_name = "/workspace/prj/study/buy_with_this_product/res_LSTM/"+ 'LSTM_ACC top1-' + str(round(int(trye),4)) + '.tar'
torch.save({
            'state_dict': net.state_dict(),
            }, save_name)

In [31]:
out.argsort(descending=True)[0]

tensor([ 717,  428,  429,  ...,  872,  909, 1333], device='cuda:0')